## Image Captioner
We explore how to build a captioning model. It will rely on the CLIP model to perform inference. We will then use `gradio` to deploy it as a web app.

## Setting up Environment

In [2]:
# Checking CUDA version.
!nvcc --version

nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2022 NVIDIA Corporation
Built on Wed_Sep_21_10:33:58_PDT_2022
Cuda compilation tools, release 11.8, V11.8.89
Build cuda_11.8.r11.8/compiler.31833905_0


In [ ]:
!conda install --yes -c pytorch pytorch=1.7.1 torchvision cudatoolkit=11.8
!pip install ftfy regex tqdm
!pip install git+https://github.com/openai/CLIP.git
!pip install gradio==4.7.1
!pip install typing_extensions==4.7.1 --upgrade

In [38]:
import clip
import torch
from PIL import Image
import gradio as gr

ImportError: ignored

In [ ]:
image = Image.open("data/dog.jpg")

In [ ]:
# Load the model
device = "cuda" if torch.cuda.is_available() else "cpu"
model, preprocess = clip.load('ViT-B/32', device)

# Download the dataset
cifar100 = CIFAR100(root=os.path.expanduser("~/.cache"), download=True, train=False)

In [ ]:
image_input = preprocess(image).unsqueeze(0).to(device)
text_inputs = torch.cat([clip.tokenize(f"a photo of a {c}") for c in cifar100.classes]).to(device)

In [ ]:
# Calculate features
with torch.no_grad():
    image_features = model.encode_image(image_input)
    text_features = model.encode_text(text_inputs)

# Pick the top 5 most similar labels for the image
image_features /= image_features.norm(dim=-1, keepdim=True)
text_features /= text_features.norm(dim=-1, keepdim=True)
similarity = (100.0 * image_features @ text_features.T).softmax(dim=-1)
values, indices = similarity[0].topk(5)

# Print the result
print("\nTop predictions:\n")
for value, index in zip(values, indices):
    print(f"{cifar100.classes[index]:>16s}: {100 * value.item():.2f}%")


Top predictions:

             fox: 18.66%
    sweet_pepper: 10.32%
            girl: 7.12%
             boy: 6.09%
            wolf: 5.54%
